In [1]:
imagename = 'GXE8'
#imagename = 'NSXT'
#imagename = 'OCVK'

imagename = 'ACP7'
imagename = 'O9FB'

# Load Image (Palette)

In [2]:
from PIL import Image, ImageFilter
from IPython.display import display
import pytesseract as tess

print(tess.get_tesseract_version())

im = Image.open(imagename + '.gif')
pix = im.load()
display(im)
print(im.size)
print(im.getbands())
print(pix[1,2])

4.0.0-beta.1
 leptonica-1.75.3
  libgif 5.1.4 : libjpeg 8d (libjpeg-turbo 1.5.2) : libpng 1.6.34 : libtiff 4.0.9 : zlib 1.2.11 : libwebp 0.6.1 : libopenjp2 2.3.0

 Found AVX2
 Found AVX
 Found SSE



(100, 30)
('P',)
251


# Convert to RGB

In [3]:
im_rgb = im.convert("RGB")
pix_rgb = im_rgb.load()
#im_rgb.save(imagename + '.rgb.gif')
display(im_rgb)
print(im_rgb.size)
print(im_rgb.getbands())
print(pix_rgb[1,2])

(100, 30)
('R', 'G', 'B')
(255, 255, 255)


# Convert to Greyscale

In [4]:
im_grey = im_rgb.convert("L")
pix_grey = im_grey.load()
#im_rgb.save(imagename + '.grey.gif')
display(im_grey)
print(pix_grey[60,27])

203


# Process 5-Pixel-Average 
(should be applied 2 times)

In [8]:
def process ( im_grey, threshold ) :
    from collections import Counter
    
    pix_grey = im_grey.load()
    
    im_processed = Image.new("RGB", im_grey.size, (255, 255, 255))
    pix_processed = im_processed.load()
    #threshold = 150

    for x in range(0,im_grey.size[0]) :
        pix_processed[x,0] = (255,255,255)
        pix_processed[x,im_grey.size[1]-1] = (255,255,255)
    for y in range(0,im_grey.size[1]) :
        pix_processed[0,y] = (255,255,255)
        pix_processed[im_grey.size[0]-1,y] = (255,255,255)

    for x in range(1,im_grey.size[0]-1) :
        for y in range(1,im_grey.size[1]-1) :
            checkval = (pix_grey[x-1,y] + pix_grey[x,y-1] + pix_grey[x,y] + pix_grey[x+1,y] + pix_grey[x,y+1]) * 0.2
            if checkval > threshold :
                pix_processed[x,y] = (255,255,255)
            else :
                pix_processed[x,y] = (1,1,1)#(int(checkval),int(checkval),int(checkval))

    display(im_processed)
    #print("recognized  6 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 6') + "'")
    #print("recognized  7 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 7') + "'")
    #print("recognized  8 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 8') + "'")
    #print("recognized 13 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 13') + "'")
    
    recognized = []
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 6').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 7').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 8').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 13').strip())
    
    im_tess = Image.new("RGB", im_processed.size, (255, 255, 255))
    im_tess.paste(im_processed)
    im_tess = im_tess.filter(ImageFilter.MedianFilter(3))
    im_tess = im_tess.convert('1')
    display(im_tess)
    #print("recognized  6 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 6') + "'")
    #print("recognized  7 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 7') + "'")
    #print("recognized  8 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 8') + "'")
    #print("recognized 13 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 13') + "'")

    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 6').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 7').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 8').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 13').strip())
    
    print( Counter(recognized) )
    return im_processed 

In [16]:
im_processed = process(im_grey, 150)
#im_processed = process(im_processed.convert("L"))
#im_processed = process(im_processed.convert("L"))
#im_processed = process(im_processed.convert("L"))

Counter({'ACR': 4, 'Nh SE? -': 2, 'ACR?': 2})


# Alternative Processing (simple threashold)

In [9]:
im_processed = im_rgb.copy()
pix_processed = im_processed.load()
threshold = 150

for x in range(1,im_rgb.size[0]-1) :
    for y in range(1,im_rgb.size[1]-1) :
        if pix_grey[x,y] > threshold :
            pix_processed[x,y] = (255,255,255)
        else :
            pix_processed[x,y] = pix_rgb[x,y]
display(im_processed)

In [10]:
!tesseract -h

Usage:
  tesseract --help | --help-extra | --version
  tesseract --list-langs
  tesseract imagename outputbase [options...] [configfile...]

OCR options:
  -l LANG[+LANG]        Specify language(s) used for OCR.
NOTE: These options must occur before any configfile.

Single options:
  --help                Show this help message.
  --help-extra          Show extra help for advanced users.
  --version             Show version information.
  --list-langs          List available languages for tesseract engine.
